<a href="https://colab.research.google.com/drive/1tMByxJ6XCVETuk8VBMaGB5jGOLXwAUFr?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XL to XS - Prompt Engineering for Smaller Models

This notebook gives you the ability to go from a large model to smaller model -- reducing costs massively while keeping quality high.

This extends the Opus to Haiku notebook ( in [`gpt-prompt-engineer`]((https://github.com/mshumer/gpt-prompt-engineer)) repo by  [Matt Shumer](https://twitter.com/mattshumer_)) to cover any large and small model combination using Portkey's [AI Gateway](https://github.com/portkey-ai/gateway)

In [ ]:
import requests

PORTKEY_API_KEY = "" # Configure your AI Gateway Key (https://app.portkey.ai/signup)

PROVIDER = "" # Any of `openai`, `anthropic`, `azure-openai`, `anyscale`, `mistral`, `gemini` and more
PROVIDER_API_KEY = "" # Enter the API key of the provider used above
LARGE_MODEL = "" # The large model to use


# If you want to use a different provider for the smaller model, uncomment these 2 lines
# SMALL_PROVIDER = "" # Any of `openai`, `anthropic`, `azure-openai`, `anyscale`, `mistral`, `gemini`
# SMALL_PROVIDER_API_KEY = ""

SMALL_MODEL = "" # The small model to use

### Portkey Client Init

Using Portkey clients for the large and small models. The gateway will allow us to make calls to any model without chaning our code.

In [ ]:
!pip install portkey_ai

In [ ]:
#@title Run this to prep the main functions

from portkey_ai import Portkey

client_large = Portkey(
    Authorization= "Bearer "+PROVIDER_API_KEY,
    provider=PROVIDER,
    api_key=PORTKEY_API_KEY,
    metadata={"_user": "gpt-prompt-engineer"},
    config={"cache": {"mode": "simple"}}
)

try:
    authorization_token = "Bearer " + SMALL_PROVIDER_API_KEY
except NameError:
    authorization_token = "Bearer " + PROVIDER_API_KEY

try:
    provider_name = SMALL_PROVIDER
except NameError:
    provider_name = PROVIDER

client_small = Portkey(
    Authorization=authorization_token,
    provider=provider_name,
    api_key=PORTKEY_API_KEY,  # Ensure this is defined and contains the correct API key.
    metadata={"_user": "gpt-prompt-engineer"},
    config={"cache": {"mode": "simple"}}
)

import json
import re

def generate_candidate_prompts(task, prompt_example, response_example):
    messages = [{
            "role": "system",
            "content":"""<task>Given an example training sample, create seven additional samples for the same task that are even better. Each example should contain a <prompt> and a <response>.</task>

<rules>
1. Ensure the new examples are diverse and unique from one another.
2. They should all be perfect. If you make a mistake, this system won't work.
</rules>

Respond in this format:
<response_format>
<example_one>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_one>

<example_two>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_two>

...
</response_format>"""
        }, {
            "role": "user",
            "content": f"""<training_task>{task}</training_task>

<prompt_example>
{prompt_example}
</prompt_example>

<response_example>
{response_example}
</response_example>"""},
    ]

    response = client_large.chat.completions.create(
        model=LARGE_MODEL,
        max_tokens=4000,
        temperature=0.5,
        messages=messages
    )
    response_text = response.choices[0]['message']['content']

    # Parse out the prompts and responses
    prompts_and_responses = []
    examples = re.findall(r'<example_\w+>(.*?)</example_\w+>', response_text, re.DOTALL)
    for example in examples:
        prompt = re.findall(r'<prompt>(.*?)</prompt>', example, re.DOTALL)[0].strip()
        response = re.findall(r'<response>(.*?)</response>', example, re.DOTALL)[0].strip()
        prompts_and_responses.append({'prompt': prompt, 'response': response})

    return prompts_and_responses

def generate_system_prompt(task, prompt_examples):
    messages = [
        {"role": "system", "content": """<your_role>Given a user-description of their <task> a set of prompt / response pairs (it'll be in JSON for easy reading) for the types of outputs we want to generate given inputs, write a fantastic system prompt that describes the task to be done perfectly.</your_role>

<rules>
1. Do this perfectly.
2. Respond only with the system prompt, and nothing else. No other text will be allowed.
</rules>

Respond in this format:
<system_prompt>
WRITE_SYSTEM_PROMPT_HERE
</system_prompt>"""
        },
        {"role": "user", "content": f"""<task>{task}</task>

<prompt_response_examples>
{str(prompt_examples)}
</prompt_response_examples>"""
        }]

    response = client_large.chat.completions.create(
        model=LARGE_MODEL,
        max_tokens=1000,
        temperature=0.5,
        messages=messages
    )
    response_text = response.choices[0]['message']['content']

    # Parse out the prompt
    system_prompt = response_text.split('<system_prompt>')[1].split('</system_prompt>')[0].strip()

    return system_prompt

def test_haiku(generated_examples, prompt_example, system_prompt):
    messages = [{"role": "system", "content": system_prompt}]

    for example in generated_examples:
      messages.append({"role": "user", "content": example['prompt']})
      messages.append({"role": "assistant", "content": example['response']})

    messages.append({"role": "user", "content": prompt_example.strip()})

    response = client_small.chat.completions.create(
        model = SMALL_MODEL,
        max_tokens=2000,
        temperature=0.5,
        messages=messages
    )
    response_text = response.choices[0]['message']['content']

    return response_text

def run_haiku_conversion_process(task, prompt_example, response_example):

    print('Generating the prompts / responses...')
    # Generate candidate prompts
    generated_examples = generate_candidate_prompts(task, prompt_example, response_example)

    print('Prompts / responses generated. Now generating system prompt...')

    # Generate the system prompt
    system_prompt = generate_system_prompt(task, generated_examples)

    print('System prompt generated:', system_prompt)


    print('\n\nTesting the new prompt on '+SMALL_MODEL+', using your input example...')
    # Test the generated examples and system prompt with the Haiku model
    small_model_response = test_haiku(generated_examples, prompt_example, system_prompt)

    print(SMALL_MODEL+' responded with:')
    print(small_model_response)

    print('\n\n!! CHECK THE FILE DIRECTORY, THE PROMPT IS NOW SAVED THERE !!')

    # Create a dictionary with all the relevant information
    result = {
        "task": task,
        "initial_prompt_example": prompt_example,
        "initial_response_example": response_example,
        "generated_examples": generated_examples,
        "system_prompt": system_prompt,
        "small_model_response": small_model_response
    }

    # Save the Haiku prompt to a Python file
    with open("haiku_prompt.py", "w") as file:
        file.write('system_prompt = """' + system_prompt + '"""\n\n')

        file.write('messages = [\n')
        for example in generated_examples:
            file.write('    {"role": "user", "content": """' + example['prompt'] + '"""},\n')
            file.write('    {"role": "assistant", "content": """' + example['response'] + '"""},\n')

        file.write('    {"role": "user", "content": """' + prompt_example.strip() + '"""}\n')
        file.write(']\n')

    return result

## Fill in your task, prompt_example, and response_example here.
Make sure you keep the quality really high here... this is the most important step!

In [ ]:
task = "refactoring complex code"

prompt_example = """def calculate_total(prices, tax, discount, shipping_fee, gift_wrap_fee, membership_discount):

    total = 0

    for i in range(len(prices)):

        total += prices[i]

    if membership_discount != 0:

        total = total - (total * (membership_discount / 100))

    if discount != 0:

        total = total - (total * (discount / 100))

    total = total + (total * (tax / 100))

    if total < 50:

        total += shipping_fee

    else:

        total += shipping_fee / 2

    if gift_wrap_fee != 0:

        total += gift_wrap_fee * len(prices)

    if total > 1000:

        total -= 50

    elif total > 500:

        total -= 25

    total = round(total, 2)

    if total < 0:

        total = 0

    return total"""

response_example = """def calculate_total(prices, tax_rate, discount_rate, shipping_fee, gift_wrap_fee, membership_discount_rate):

    def apply_percentage_discount(amount, percentage):

        return amount * (1 - percentage / 100)

    def calculate_shipping_fee(total):

        return shipping_fee if total < 50 else shipping_fee / 2

    def apply_tier_discount(total):

        if total > 1000:

            return total - 50

        elif total > 500:

            return total - 25

        return total

    subtotal = sum(prices)

    subtotal = apply_percentage_discount(subtotal, membership_discount_rate)

    subtotal = apply_percentage_discount(subtotal, discount_rate)



    total = subtotal * (1 + tax_rate / 100)

    total += calculate_shipping_fee(total)

    total += gift_wrap_fee * len(prices)



    total = apply_tier_discount(total)

    total = max(0, round(total, 2))



    return total"""

### Now, let's run this system and get our new prompt!
At the end, you'll see a new file pop up in the directory that contains everything you'll need to reduce your costs while keeping quality high w/ Haiku!

In [ ]:
result = run_haiku_conversion_process(task, prompt_example, response_example)

Generating the prompts / responses...
Prompts / responses generated. Now generating system prompt...
System prompt generated: You are an expert code refactoring assistant. Your task is to take a given piece of code and refactor it to be more concise, efficient, and maintainable while preserving its original functionality. Focus on improving code readability, eliminating redundancies, optimizing performance, and applying best practices and design patterns where appropriate. Provide a clear, refactored version of the code that showcases your expertise in writing clean, high-quality code.


Testing the new prompt on claude-3-haiku-20240307, using your input example...
claude-3-haiku-20240307 responded with:
def calculate_total(prices, tax, discount, shipping_fee, gift_wrap_fee, membership_discount):
    subtotal = sum(prices)
    
    if membership_discount:
        subtotal *= (1 - membership_discount / 100)
    
    if discount:
        subtotal *= (1 - discount / 100)
    
    total = 

### View logs on Portkey
Go to the logs tab in Portkey to inspect the 3 calls made and the results returned. Note that cache is enabled in the calls, so all calls after the first one would return instantaneously.